# Numerical Analysis - IMPA 2020
### Professor Dan Marchesin
### Hallison Paz, 1st year Phd student
### Pedro Fonini, 2nd year MSc student

# Modeling epidemics with SIR model

In [ ]:
import os

from ipywidgets import interact, interact_manual

import numpy as np
from scipy import linalg, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerTuple
mpl.rc('font', size=18)

In [ ]:
OUTPUT_DIR = os.path.join('images', 'SIR')

## Newton's method

In [ ]:
class IterationLimitError(Exception):
    """Exception type for too many iterations.
    
    Signals that we have reached the maximum allowed number of
    iterations for an iterative numerical method."""

In [ ]:
def newton(f, df, err, x0, max_iter=10,
           max_iter_action='fail', verbose=False):
    """Newton's method for solving the equation f(x)=0.
    
    x0: initial guess; shape (N,)
    f: callable; vector valued function of x.
        input: shape (N,)
        output: shape (N,)
    df: callable; for each x, returns the jacobian matrix of f at x.
        input: shape (N,)
        output: shape (N, N)
    err: callable; for each x, return the expected error in the
            numerical evaluation of f(x).
        input: shape (N,)
        output: shape (N,), non-negative entries
    max_iter: integer; the maximum allowed number of iterations
    max_iter_action: action to be taken upon reaching the maximum number
            of iterations. 'return' if we should return the best guess
            up to now; or 'fail' if we should not return and raise an
            expection.
    """
    f_value = f(x0)

    for n_iter in range(max_iter):
        v = linalg.solve(df(x0), f_value)
        x0 = x0 - v
        f_value = f(x0)
        if verbose:
            print(f'n    = {n_iter+1}')
            print(f'x    = {x0}')
            print(f'step = {np.linalg.norm(v)}')
            print(f'f    = {f_value}')
            print(f'err  = {err(x0)}')
            print()
        if np.all(np.abs(f_value) <= err(x0)):
            # f(x0) is so small that it could already be zero, it's
            # impossible to know, so we might as well stop here.
            if verbose:
                print(f'success after n_iter={n_iter+1}')
            return x0

    if max_iter_action == 'fail':
        raise IterationLimitError(
            "Maximum number of iterations reached and still |f|>err:\n"
            f"    max_iter = {max_iter}\n"
            f"    x        = {x0}\n"
            f"    step     = {np.linalg.norm(v)}\n"
            f"    f        = {f_value} +/- {err(x0)}")

### Testing Newton's method:

In [ ]:
def f(x):
    return x**2 - 2

def df(x):
    return 2*x

eps = np.finfo(float).eps
def err(x):
    # the value calculated is (x²(1+eps) - 2)(1+eps) =
    # = (x²-2) + O( 2 * (x²+1) * eps )
    return 2 * (x**2 + 1) * eps

sqrt2 = newton(f, df, err, x0=2, verbose=True)
print(sqrt2)
print(sqrt2 - np.sqrt(2))

In [ ]:
sqrt2 = newton(f, df, err, x0=10, verbose=True)
print(sqrt2)
print(sqrt2 - np.sqrt(2))

In [ ]:
def err(x):
    return eps

sqrt2 = newton(f, df, err, x0=2, verbose=True)
print(sqrt2)
print(sqrt2 - np.sqrt(2))

In [ ]:
def f(x):
    # gradient of the rosenbrok function
    yx2 = x[1] - x[0]**2
    return [2*(x[0] - 1) - 400*x[0]*yx2,
            200 * yx2]

def df(x):
    # Hessian of rosenbrok
    Rxx = 2 + 400*(3*x[0]**2 - x[1])
    Rxy = -400 * x[0]
    Ryy = 200
    return [[Rxx, Rxy], [Rxy, Ryy]]

def err(x):
    return [4*(1 + x[0] + 400*x[0]*(x[0]**2+x[1])) * eps,
            (3*x[0]**2 + 2*x[1]) * eps]

minimum = newton(f, df, err, x0=[7, 13], verbose=True)
print(minimum)
print(minimum - [1,1])

In [ ]:
minimum = newton(f, df, err, x0=[10, 100], verbose=True)
print(minimum)
print(minimum - [1,1])

### Implicit Euler method

$$\begin{gather}
f(S,I,R) = (-\beta S I, \beta SI - \gamma I, \gamma I) \qquad(1)\\
(S_1,I_1,R_1) = (S, I, R) + dt \cdot f(S_1,I_1,R_1) \qquad(2)\\
f=(f_S, f_I, f_R) \qquad(3)\\
\begin{bmatrix}
S_1 - S - dt\cdot f_S(S_1,I_1,R_1) \\
I_1 - I - dt\cdot f_I(S_1,I_1,R_1) \\
R_1 - R - dt\cdot f_R(S_1,I_1,R_1)
\end{bmatrix} = 0 \qquad(4)\\
\end{gather}$$

In [ ]:
def implicit_euler(f, J, err, x0, dt, N, verbose_newton=False):
    # f: the function in the diff. equation x' = f(x)
    # J: the jacobian of f
    # x0: initial point
    # dt: time step
    # N: number of samples (i.e. number of time steps)
    dim = len(x0)  # dimension of phase space
    x = np.empty((N+1, dim))
    x[0] = x0
    for i in range(N):
        def implicit_eq(v):
            # 'v' stands for x[i+1]
            return v - x[i] - dt*f(v)
        def jacobian_eq(v):
            return np.eye(dim) - dt*J(v)
        def err_eq(v):
            return (np.abs(v) + np.abs(x[i]) +
                    np.abs(dt*f(v)))*2*eps + \
                    dt*err(v)
        if verbose_newton:
            print('\n\n\n==================\n')
        x[i+1] = newton(implicit_eq, jacobian_eq, err_eq, x0=x[i],
                        verbose=verbose_newton)
    return x

### Apply the method to SIR

In [ ]:
beta = 2.5
gamma = 0.8

def SIR(I0=0.0001,
        dt=1/24,  # default time step = 1 hour
        days=20,
        beta=beta,
        gamma=gamma,
        verbose_newton=False):
    
    def f(x):
        S, I, R = x
        return np.array([-beta*S*I,
                          beta*S*I - gamma*I,
                          gamma*I])
    def J(x):
        S, I, R = x
        return np.array([[-beta*I, -beta*S,         0],
                         [ beta*I,  beta*S - gamma, 0],
                         [ 0,       gamma,          0]])
    def err(x):
        S, I, R = x
        return np.array([2*beta*S*I,
                         3*beta*S*I + 2*gamma*I,
                         gamma*I]) * eps
    
    n_samples = int(days//dt)
    t = np.arange(n_samples + 1) * dt
    x0 = np.array([1 - I0, I0, 0])
    
    x = implicit_euler(f, J, err, x0, dt, n_samples,
                       verbose_newton=verbose_newton)
    return t, x

In [ ]:
# Testing
t, x = SIR(days=5/24, verbose_newton=True)

In [ ]:
%%time
t, x = SIR(dt=1/24)

In [ ]:
%%time

# Generate another solution, from the built-in scipy solver

def f(_, x):
    S, I, R = x
    return np.array([-beta*S*I,
                      beta*S*I - gamma*I,
                      gamma*I])
def J(_, x):
    S, I, R = x
    return np.array([[-beta*I, -beta*S,         0],
                     [ beta*I,  beta*S - gamma, 0],
                     [ 0,       gamma,          0]])

solver = integrate.ode(f, J)

x_scipy = np.empty(x.shape)
x_scipy[0] = x[0]
solver.set_initial_value(x[0]);

for i in range(1, len(x_scipy)):
    if not solver.successful():
        solver_vars = f'{solver.t=} {solver.y=}'
        raise RuntimeError(f'ODE solver error: {solver_vars}')
    x_scipy[i] = solver.integrate(t[i])

In [ ]:
fig, ax = plt.subplots(3, 1, sharex=True, figsize=(10,21))

l = ax[0].plot(t, x)
ax[0].set_prop_cycle(None)  # reset color cycle
l_scipy = ax[0].plot(t, x_scipy, ':')
ax[0].legend([tuple(l), tuple(l_scipy)],
          ["S, I, R (implicit euler)", "S, I, R (scipy)"],
          handler_map={tuple: HandlerTuple(ndivide=None)})
ax[0].grid()

l = ax[1].plot(t, x-x_scipy)
l_norm = ax[1].plot(t, np.linalg.norm(x-x_scipy, axis=1), color='k')
ax[1].grid()
ax[1].legend([tuple(l), tuple(l_norm)],
          ["S, I, R errors", "total (norm of) error"],
          handler_map={tuple: HandlerTuple(ndivide=None)})
ax[1].set_title("absolute error")

l = ax[2].plot(t, x/x_scipy-1)
l_norm = ax[2].plot(t, np.linalg.norm(x-x_scipy, axis=1) / \
                       np.linalg.norm(x_scipy, axis=1),
                    color='k')
ax[2].grid()
ax[2].legend([tuple(l), tuple(l_norm)],
          ["S, I, R errors", "total (norm of) error"],
          handler_map={tuple: HandlerTuple(ndivide=None)})
ax[2].set_title("relative error");

## Plotting Curves for multiple values of Beta and Gamma

In [ ]:
@interact_manual(I0=(0.0, 1.0, 0.01), 
                 dt=(0.1, 2))
def draw_SIR_curves(I0 = 0.11,
                    dt = 0.1,
                    days = 20,
                    beta = 2.5,
                    gamma = 0.8):
    t, x = SIR(I0, dt, days, beta, gamma)
    S, I, R = x.T
    t = [dt*i for i in range(len(S))]
    fig = plt.figure(figsize=(12,8))
    l1, l2, l3 = plt.plot(t, S, t, I, t, R)
    fig.legend((l1, l2, l3), ('S', 'I', 'R'), 'center right')
    plt.xlabel('Time')
    plt.ylabel('% population')
    plt.show()

## Ploting orbits in the triangle S + I <= 1 (Autonomous system of ODE)

In [ ]:
@interact_manual(samples=(2, 50, 1), 
                 dt=(0.1, 2),
                days=20)
def draw_SI_relation(samples = 11,
                    dt = 0.1,
                    days = 20,
                    beta = 2.5,
                    gamma = 0.8):
    s_list = []
    i_list = []
    for i0 in np.linspace(0, 1.0, samples):
        t, x = SIR(i0, dt, days, beta, gamma)
        S, I, R = x.T
        s_list.append(S)
        i_list.append(I)
        

    fig = plt.figure(figsize=(10, 10))
    plt.plot([0.0, 1.0], [1.0, 0.0], 'gray')
    for k in range(len(s_list)):
        plt.plot(s_list[k], i_list[k], 'blue' if k%2 else '#0000ff80')
        
    for i0 in np.linspace(0.0, 0.1, int(samples/2)):
        t, x = SIR(i0, dt, days, beta, gamma)
        S, I, R = x.T
        plt.plot(S, I, 'red' if k%2 else '#ff000080')
        
    plt.xlabel('Susceptible')
    plt.ylabel('Infected')
    plt.show()

## Comparing Euler Method with and without Predictor-Corrector technique

(TODO: compare all three: euler implicit, euler predictor-corrector, and eu

In [ ]:
@interact_manual(I0=(0.0, 1.0, 0.01), 
                 dt=(0.1, 2),
                 show=['all', 'S', 'I', 'R'])
def compare_methods(I0 = 0.11,
                    dt = 1,
                    days = 20,
                    beta = 2.5,
                    gamma = 0.8,
                    show='all'):
    
    fe_sir = SIR(I0, dt, days, beta, gamma, False)
    pc_sir = SIR(I0, dt, days, beta, gamma, True)
    t =  [dt*i for i in range(len(fe_sir[0]))]
    legends =['S', 'I', 'R']
    
    fig, ax = plt.subplots(1, 3, figsize=(28,8)) if show == 'all' else (plt.figure(figsize=(16,8)), None)
    if show == 'all':
        for index in range(len(fe_sir)):
            ax[index].plot(t, fe_sir[index], label='{} FE'.format(legends[index]))
            ax[index].plot(t, pc_sir[index], label='{} P-C'.format(legends[index]))
            ax[index].legend(loc= 'center right')
    else:
        index = legends.index(show)
        plt.plot(t, fe_sir[index], label='{} FE'.format(legends[index]))
        plt.plot(t, pc_sir[index], label='{} P-C'.format(legends[index]))
        fig.legend(loc= 'center right')
        
    fig.suptitle('Comparinson between methods for ODE')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.show()
    